<a href="https://colab.research.google.com/github/zartuyt/DataExtractor/blob/main/DataExtractor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
!git clone https://github.com/zartuyt/DataExtractor

Cloning into 'DataExtractor'...
remote: Enumerating objects: 7, done.
remote: Counting objects: 100% (7/7), done.
remote: Compressing objects: 100% (7/7), done.
remote: Total 7 (delta 1), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (7/7), 9.56 KiB | 9.56 MiB/s, done.
Resolving deltas: 100% (1/1), done.


In [6]:
import os
os.chdir('DataExtractor')

In [7]:
invoices_new_path = 'invoices_new.pkl'
expired_invoices_path = 'expired_invoices.txt'

In [18]:
import pandas as pd
import pickle

class DataExtractor:
    def __init__(self, invoices_new_path, expired_invoices_path):
        # Initialize with paths to the new invoices and expired invoices files
        self.invoices_new_path = invoices_new_path
        self.expired_invoices_path = expired_invoices_path
        self.type = {0: 'Material', 1: 'Equipment', 2: 'Service', 3: 'Other'}

    def load_new_invoices(self):
        # Load the new invoices from a pickle file
        with open(self.invoices_new_path, 'rb') as file:
            self.data = pickle.load(file)

    def load_expired_invoices(self):
        # Load the expired invoices from a text file, spritting any commas and convert them to a set of integers
        with open(self.expired_invoices_path, 'r') as file:
            self.expired_invoices = set(map(int, map(str.strip, file.read().split(','))))

    def convert_to_int(self, value):
        # Handle an error in case value cannot be converted to int
        try:
            return int(value)
        except ValueError:
            return None

    def transform_data(self):
        # Prepare a list to store the flattened data
        flattened_data = []

        # Iterate over each invoice in the data
        for invoice in self.data:
            # Convert the invoice ID to an integer, filtering out any non-digit characters
            invoice_id_str = str(invoice['id'])
            invoice_id = int(''.join(filter(str.isdigit, invoice_id_str)))

            # Handle exceptions for invalid dates
            try:
                created_on = pd.to_datetime(invoice['created_on'])
            except (ValueError, pd.errors.OutOfBoundsDatetime):
                continue  # If the date is invalid just skip

            # Check if the 'items' key exists in the invoice
            if 'items' not in invoice:
                continue  # If there are no items skip this invoice

            # Calculate the total price of the invoice
            invoice_total = sum(
                self._parse_price(item['item']['unit_price']) * self._parse_quantity(item['quantity'])
                for item in invoice['items']
            )

            # Process each item in the invoice with the coloumns in the following order
            for item in invoice['items']:
                invoiceitem_id = item['item']['id']
                invoiceitem_name = item['item']['name']
                type_ = self._parse_type(item['item']['type'])
                unit_price = self._parse_price(item['item']['unit_price'])
                quantity = self._parse_quantity(item['quantity'])
                total_price = unit_price * quantity
                percentage_in_invoice = total_price / invoice_total
                is_expired = invoice_id in self.expired_invoices

                # Add the  data to the list
                flattened_data.append({
                    'invoice_id': invoice_id,
                    'created_on': created_on,
                    'invoiceitem_id': invoiceitem_id,
                    'invoiceitem_name': invoiceitem_name,
                    'type': type_,
                    'unit_price': unit_price,
                    'total_price': total_price,
                    'percentage_in_invoice': percentage_in_invoice,
                    'is_expired': is_expired
                })

        # Convert the flattened data to a DataFrame
        flat_df = pd.DataFrame(flattened_data)
        # Sort the DataFrame by invoice ID and item ID using built-in function
        flat_df = flat_df.sort_values(by=['invoice_id', 'invoiceitem_id'])
        return flat_df

    def _parse_price(self, price):
        # Handle error in case price is not convertable to int
        try:
            return int(price)
        except ValueError:
            return 0

    def _parse_quantity(self, quantity):
        # Handle exception in case price is not convertable to int
        try:
            return int(quantity)
        except ValueError:
            return 0

    def _parse_type(self, type_):
        # Handle exception in case if conversion to readable string fails
        try:
            return self.type[int(type_)]
        except (ValueError, KeyError):
            return 'Other'

    def run(self):
        # Load the invoices
        self.load_new_invoices()
        self.load_expired_invoices()
        transformed_data = self.transform_data()
        transformed_data.to_csv('transformed_invoices.csv', index=False)

        # Reading the contents of the newly created CSV file
        print("Transformed Invoices CSV Content:\n")
        print(transformed_data.to_string(index=False))

        return "CSV file saved and printed successfully."

# Create an instance of the DataExtractor
extractor = DataExtractor(invoices_new_path, expired_invoices_path)

result = extractor.run()

Transformed Invoices CSV Content:

 invoice_id created_on  invoiceitem_id invoiceitem_name      type  unit_price  total_price  percentage_in_invoice  is_expired
     301695 2019-04-26          103215        ii_103215     Other         135          945               0.374851       False
     301695 2019-04-26          166227        ii_166227 Equipment         118          708               0.280841       False
     301695 2019-04-26          171394        ii_171394     Other         128          128               0.050774       False
     301695 2019-04-26          195625        ii_195625     Other         148          740               0.293534       False
     304245 2019-03-17          121446        ii_121446   Service         158          632               0.128429       False
     304245 2019-03-17          135235        ii_135235   Service         151          906               0.184109       False
     304245 2019-03-17          143674        ii_143674   Service         176      